In [91]:
from astropy.table import Table

In [92]:
# load table
fpath = '../'
fname = 'object_sdss_tiling.fits'
t_1 = Table.read(fpath + fname)

# number of rows
print('number of rows in table: {0}'.format(len(t_1)))

# print column names
for name in sorted(t_1.colnames):
    print(name)

number of rows in table: 2506754
CAMCOL
COLLISIONGROUP
DEC
FIELD
ID
MASK
OBJC_COLC
OBJC_ROWC
OVERLAP
PRIMTARGET
PRIORITY
PSFMAG
RA
RERUN
RUN
SDSS_TILING_FIRSTGROUP
SDSS_TILING_INGROUP
SDSS_TILING_MULTGROUP
SDSS_TILING_NEXTGROUP
SDSS_TILING_TAG
SDSS_TILING_TAG_PRIMARY
SECTARGET
TILEID


In [150]:
# load table
fpath = '../'
fname = 'lss_index.dr72.fits'
t_2 = Table.read(fpath + fname)

# number of rows
print('number of rows in table: {0}'.format(len(t_2)))

# print column names
for name in sorted(t_2.colnames):
    print(name)

number of rows in table: 2506754
DEC
FGOTMAIN
ICOMB
ILSS
ISBAD
M
MMAX
QUALITY
RA
SECTOR
VAGC_SELECT
Z
ZTYPE


In [151]:
# load table
fpath = '../'
fname = 'object_sdss_imaging.fits'
t_3 = Table.read(fpath + fname)

# number of rows
print('number of rows in table: {0}'.format(len(t_3)))

# print column names
for name in sorted(t_3.colnames):
    print(name)

number of rows in table: 2506754
BALKAN_ID
CALIBOBJ_POSITION
CALIB_STATUS
CAMCOL
COLC
DEC
DEVFLUX
DEVFLUX_IVAR
EXPFLUX
EXPFLUX_IVAR
EXTINCTION
FIBERFLUX
FIELD
FLAGS
FLAGS2
FRACPSF
ID
IFIELD
MODELFLUX
MODELFLUX_IVAR
NCHILD
NMGYPERCOUNT
OBJC_COLC
OBJC_FLAGS
OBJC_FLAGS2
OBJC_ROWC
PETROFLUX
PETROFLUX_IVAR
PETROR50
PETROR90
PSFFLUX
PSFFLUX_IVAR
PSP_SKYFLUX
RA
RERUN
RESOLVE_STATUS
RUN
SCORE
SDSS_IMAGING_TAG
SDSS_IMAGING_TAG_PRIMARY
SKYFLUX
VAGC_SELECT


In [152]:
# load table from Eyal Kazin
# http://cosmo.nyu.edu/~eak306/SDSS-LRG.html

names=['ra', 'dec', 'z', 'M_g', 'sector_completeness', 'n(z)*1e4', 'radial_weight',
       'fiber_coll_weight (>=1)', 'fogtmain', 'ilss', 'icomb', 'sector']
t_4 = Table.read('../DR7-Full.ascii', format='ascii', names=names)
print(len(t_4))

105831


In [153]:
# LRG selection mask
# see table 27 in Stoughton + (2002)
mask_a = (t_1['PRIMTARGET'] & 32) > 0
mask_b = (t_1['PRIMTARGET'] & 2**31) == 0  # account for stripe 82 selection
mask_1 = (mask_a & mask_b)

In [154]:
# redshift selection
# sdss redshift
mask_a = (t_2['ZTYPE'] == 1)

# redshift range
zmin, zmax = (0.16, 0.47)
mask_b = (t_2['Z'] >=zmin) & (t_2['Z'] <=zmax)

# not in star mask
mask_c = (t_2['ICOMB'] != -1)

mask_2 = (mask_a & mask_b) & mask_c

In [159]:
# completeness cut

t_by_sector = t_2.group_by('SECTOR')
sectors = np.unique(t_2['SECTOR'])

selection_func = lambda x: np.sum(x==1)
num_observed = t_by_sector['ZTYPE'].groups.aggregate(selection_func)

selection_func = lambda x: len(x)
num_targets = t_by_sector['ZTYPE'].groups.aggregate(selection_func)

sector_completness = 1.0*num_observed/num_targets

mask_a = (sector_completness >= 0.6)
sectors_to_keep = sectors[mask_a]

# use sectors from Eyal Kazin's catalog
sectors_to_keep = np.unique(t_4['sector'])

mask_3 = np.in1d(np.array(t_2['SECTOR']), sectors_to_keep)


In [164]:
from astropy.cosmology import FlatLambdaCDM
from scipy import interpolate

cosmo = FlatLambdaCDM(H0=70, Om0=0.25)

def interpolated_distmod(z, cosmo):
    
    z = np.atleast_1d(z)
    
    z_sample = np.linspace(0.000001, 1.0, 1000)
    d = cosmo.distmod(z_sample)
    f = interpolate.InterpolatedUnivariateSpline(z_sample, d, k=1, ext=3, check_finite=True)
    
    return f(z)
    

In [165]:
# magnitude cut
from astro_utils.magnitudes import color_k_correct

mr=22.5-2.5*np.log10(t_3['PETROFLUX'][:,2])-t_3['EXTINCTION'][:,2] 

z = t_2['Z']
delta_g, u_minus_g, g_minus_r, r_minus_i = color_k_correct(z, galaxy_type = 'non-star-forming')
z_calibration = 0.2

Mg = mr - interpolated_distmod(z, cosmo) - delta_g + g_minus_r - z_calibration
Mg = Mg - 5.0*np.log10(cosmo.h)
 
mask_4 = (Mg >= -23.2) & (Mg <= -21.2)

/Users/duncan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log10
  after removing the cwd from sys.path.


In [166]:
mask = (mask_1 & mask_2) & (mask_3 & mask_4)

In [163]:
np.sum(mask), np.sum(mask_1), np.sum(mask_2), np.sum(mask_3), np.sum(mask_4)

(105838, 214124, 247033, 1555258, 180076)

In [172]:
# save indices of LRG sample

N = len(t_1)

inds = np.arange(0,N).astype('int')
inds = inds[mask]

fpath = '../'
fname = 'lrg_sample_indices'
np.save(fpath+fname, inds)

In [178]:
runs = np.unique(t_1['RUN'])

files = []
for run in runs:
    mask = t_1['RUN']==run
    camcols = np.unique(t_1['CAMCOL'][mask])
    for camcol in camcols:
        files.append((run, camcol))

In [184]:
for p in files:
    print('calibObj-'+str(p[0]).zfill(6)+'-'+str(p[1])+'.fits')

calibObj-000000-0.fits
calibObj-000094-1.fits
calibObj-000094-2.fits
calibObj-000094-3.fits
calibObj-000094-4.fits
calibObj-000094-5.fits
calibObj-000094-6.fits
calibObj-000125-1.fits
calibObj-000125-2.fits
calibObj-000125-3.fits
calibObj-000125-4.fits
calibObj-000125-5.fits
calibObj-000125-6.fits
calibObj-000745-1.fits
calibObj-000745-2.fits
calibObj-000745-3.fits
calibObj-000745-4.fits
calibObj-000745-5.fits
calibObj-000745-6.fits
calibObj-000752-1.fits
calibObj-000752-2.fits
calibObj-000752-3.fits
calibObj-000752-4.fits
calibObj-000752-5.fits
calibObj-000752-6.fits
calibObj-000756-1.fits
calibObj-000756-2.fits
calibObj-000756-3.fits
calibObj-000756-4.fits
calibObj-000756-5.fits
calibObj-000756-6.fits
calibObj-001033-1.fits
calibObj-001033-2.fits
calibObj-001033-3.fits
calibObj-001033-4.fits
calibObj-001033-5.fits
calibObj-001033-6.fits
calibObj-001035-1.fits
calibObj-001035-2.fits
calibObj-001035-3.fits
calibObj-001035-4.fits
calibObj-001035-5.fits
calibObj-001035-6.fits
calibObj-00

In [187]:
# load table
fpath = '../parameters/'
fname = 'calibObj-000094-3.fits'
t_1 = Table.read(fpath + fname)

# number of rows
print('number of rows in table: {0}'.format(len(t_1)))

# print column names
for name in sorted(t_1.colnames):
    print(name)

number of rows in table: 1
AB_DEV
AB_DEVERR
AB_EXP
AB_EXPERR
AIRMASS
APERFLUX
APERFLUX_IVAR
BALKAN_ID
CALIB_STATUS
CAMCOL
CLOUDCAM
COLC
COLCERR
COLV
COLVERR
DEC
DEVFLUX
DEVFLUX_IVAR
DEV_LNL
EXPFLUX
EXPFLUX_IVAR
EXP_LNL
EXTINCTION
FIBERFLUX
FIBERFLUX_IVAR
FIELD
FIRST_DEC
FIRST_FINT
FIRST_FMAJ
FIRST_FMIN
FIRST_FPA
FIRST_FPEAK
FIRST_MAJ
FIRST_MATCHDIST
FIRST_MIN
FIRST_NMATCH
FIRST_PA
FIRST_RA
FIRST_RMS
FIRST_SKYRMS
FIRST_WARNING
FLAGS
FLAGS2
FRACPSF
ID
IFIELD
ISO_A
ISO_AERR
ISO_AGRAD
ISO_B
ISO_BERR
ISO_BGRAD
ISO_COLC
ISO_COLCERR
ISO_COLCGRAD
ISO_PHIGRAD
ISO_ROWC
ISO_ROWCERR
ISO_ROWCGRAD
MJD
MODELFLUX
MODELFLUX_IVAR
M_CR4
M_CR4_PSF
M_E1
M_E1E1ERR
M_E1E2ERR
M_E1_PSF
M_E2
M_E2E2ERR
M_E2_PSF
M_RR_CC
M_RR_CCERR
M_RR_CC_PSF
NCHILD
NDETECT
NEDGE
NMGYPERCOUNT
NMGYPERCOUNT_IVAR
NOBSERVE
NPROF
OBJC_COLC
OBJC_COLCERR
OBJC_FLAGS
OBJC_FLAGS2
OBJC_PROB_PSF
OBJC_ROWC
OBJC_ROWCERR
OBJC_TYPE
OFFSETDEC
OFFSETRA
PARENT
PETROFLUX
PETROFLUX_IVAR
PETROR50
PETROR50ERR
PETROR90
PETROR90ERR
PETRORAD
PETRORADERR
P

In [190]:
t_1

RUN,RERUN,CAMCOL,FIELD,ID,PARENT,NCHILD,OBJC_TYPE,OBJC_PROB_PSF,OBJC_FLAGS,OBJC_FLAGS2,OBJC_ROWC,OBJC_ROWCERR,OBJC_COLC,OBJC_COLCERR,ROWV,ROWVERR,COLV,COLVERR,ROWC [5],ROWCERR [5],COLC [5],COLCERR [5],PETRORAD [5],PETRORADERR [5],PETROR50 [5],PETROR50ERR [5],PETROR90 [5],PETROR90ERR [5],Q [5],QERR [5],U [5],UERR [5],M_E1 [5],M_E2 [5],M_E1E1ERR [5],M_E1E2ERR [5],M_E2E2ERR [5],M_RR_CC [5],M_RR_CCERR [5],M_CR4 [5],M_E1_PSF [5],M_E2_PSF [5],M_RR_CC_PSF [5],M_CR4_PSF [5],ISO_ROWC [5],ISO_ROWCERR [5],ISO_ROWCGRAD [5],ISO_COLC [5],ISO_COLCERR [5],ISO_COLCGRAD [5],ISO_A [5],ISO_AERR [5],ISO_AGRAD [5],ISO_B [5],ISO_BERR [5],ISO_BGRAD [5],ISO_PHIGRAD [5],R_DEV [5],R_DEVERR [5],AB_DEV [5],AB_DEVERR [5],R_EXP [5],R_EXPERR [5],AB_EXP [5],AB_EXPERR [5],STAR_LNL [5],EXP_LNL [5],DEV_LNL [5],FRACPSF [5],FLAGS [5],FLAGS2 [5],TYPE [5],PROB_PSF [5],NPROF [5],TAI [5],PSP_STATUS [5],PIXSCALE [5],RA,DEC,OFFSETRA [5],OFFSETDEC [5],PSF_FWHM [5],MJD,AIRMASS [5],PHI_OFFSET [5],PHI_ISO_DEG [5],PHI_DEV_DEG [5],PHI_EXP_DEG [5],EXTINCTION [5],SKYFLUX [5],SKYFLUX_IVAR [5],PSFFLUX [5],PSFFLUX_IVAR [5],FIBERFLUX [5],FIBERFLUX_IVAR [5],MODELFLUX [5],MODELFLUX_IVAR [5],PETROFLUX [5],PETROFLUX_IVAR [5],DEVFLUX [5],DEVFLUX_IVAR [5],EXPFLUX [5],EXPFLUX_IVAR [5],"APERFLUX [5,8]","APERFLUX_IVAR [5,8]",CLOUDCAM [5],CALIB_STATUS [5],NMGYPERCOUNT [5],NMGYPERCOUNT_IVAR [5],FIRST_RA,FIRST_DEC,FIRST_WARNING,FIRST_FINT,FIRST_FPEAK,FIRST_RMS,FIRST_MAJ,FIRST_MIN,FIRST_PA,FIRST_FMAJ,FIRST_FMIN,FIRST_FPA,FIRST_SKYRMS,FIRST_MATCHDIST,FIRST_NMATCH,TMASS_RA,TMASS_DEC,TMASS_ERR_MAJ,TMASS_ERR_MIN,TMASS_ERR_ANG,TMASS_J,TMASS_J_IVAR,TMASS_H,TMASS_H_IVAR,TMASS_K,TMASS_K_IVAR,TMASS_PH_QUAL,TMASS_RD_FLG,TMASS_BL_FLG,TMASS_CC_FLG,TMASS_GAL_CONTAM,TMASS_MP_FLG,TMASS_JDATE,TMASS_DUP_SRC,TMASS_MATCHDIST,TMASS_NMATCH,RESOLVE_STATUS,THING_ID,IFIELD,BALKAN_ID,NOBSERVE,NDETECT,NEDGE,VAGC_SELECT,PSP_SKYFLUX [5],SCORE
int32,str3,int32,int32,int32,int32,int32,int32,float32,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,float32,int32,float64,int32,float32,float64,float64,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32,float32,float64,float64,str1,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,str3,int16,int16,str3,uint8,uint8,float64,uint8,float32,int16,int16,int32,int32,int32,int16,int16,int16,int32,float32,float32
94,137,3,159,39,38,0,3,0.0,268570672,8208,873.7111,0.0045597325,1976.9955,0.003936762,0.0073938603,0.019512802,-0.00353231,0.018940281,881.11053 .. 883.9141,0.06508728 .. 0.012001444,1980.2483 .. 1978.4902,0.052453935 .. 0.008990291,11.408673 .. 9.785451,0.63752365 .. 0.59095,5.217966 .. 4.4263263,0.11814165 .. 0.054004908,13.538366 .. 12.304064,0.8675723 .. 0.43159723,-0.06384066 .. -0.087312505,0.008593687 .. 0.002176076,0.12334547 .. 0.14897448,0.008688115 .. 0.002207084,-0.15271015 .. -0.21055932,0.28923962 .. 0.30327877,0.012589573 .. 0.0025420017,0.0026773044 .. 0.0006570997,0.012194482 .. 0.0024778284,25.875473 .. 15.816067,0.35285676 .. 0.045223415,2.2280245 .. 2.2736182,-0.13054778 .. -0.29520085,-0.046564847 .. -0.06575975,7.706025 .. 5.487757,2.0634081 .. 2.0727525,880.96295 .. 883.5489,-1000.0 .. -1000.0,-0.09278195 .. -0.19187364,1980.3134 .. 1978.6132,-1000.0 .. -1000.0,-0.15805827 .. -0.0733293,29.644596 .. 36.052837,-1

In [189]:
t_3['CALIBOBJ_POSITION']

0
0
1
2
3
4
5
6
7
8
9
